<a href="https://colab.research.google.com/github/bsultani047-cell/colab/blob/main/Welcome_to_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#!pip install datasets bitsandbytes trl
# bitsandbytes had to be bumped to 0.45.2 to avoid errors in Colab env
!pip install transformers==4.46.2 peft==0.13.2 accelerate==1.1.1 trl==0.12.1 bitsandbytes==0.45.2 datasets==3.1.0 huggingface-hub==0.26.2 safetensors==0.4.5 pandas==2.2.2 matplotlib==3.8.0 numpy==1.26.4


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 87.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 333.2/333.2 kB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.9/310.9 kB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 34.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.5/447.5 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 435.0/435.0 kB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 85.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 74.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [51]:
import os
import torch
from datasets import load_dataset
from peft import get_peft_model, LoraConfig, prepare_model_for_kbit_training
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from trl import SFTConfig, SFTTrainer
from datasets import Dataset


In [2]:
bnb_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_quant_type="nf4",
   bnb_4bit_use_double_quant=True,
   bnb_4bit_compute_dtype=torch.float32
)
repo_id = 'microsoft/Phi-3-mini-4k-instruct'
model = AutoModelForCausalLM.from_pretrained(
   repo_id, device_map="cuda:0", quantization_config=bnb_config
)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

In [3]:
print(model.get_memory_footprint()/1e6)

2206.347264


In [4]:
model

Phi3ForCausalLM(
  (model): Phi3Model(
    (embed_tokens): Embedding(32064, 3072, padding_idx=32000)
    (embed_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-31): 32 x Phi3DecoderLayer(
        (self_attn): Phi3SdpaAttention(
          (o_proj): Linear4bit(in_features=3072, out_features=3072, bias=False)
          (qkv_proj): Linear4bit(in_features=3072, out_features=9216, bias=False)
          (rotary_emb): Phi3RotaryEmbedding()
        )
        (mlp): Phi3MLP(
          (gate_up_proj): Linear4bit(in_features=3072, out_features=16384, bias=False)
          (down_proj): Linear4bit(in_features=8192, out_features=3072, bias=False)
          (activation_fn): SiLU()
        )
        (input_layernorm): Phi3RMSNorm((3072,), eps=1e-05)
        (resid_attn_dropout): Dropout(p=0.0, inplace=False)
        (resid_mlp_dropout): Dropout(p=0.0, inplace=False)
        (post_attention_layernorm): Phi3RMSNorm((3072,), eps=1e-05)
      )
    )
    (norm): Phi3RMSNorm((3072

In [5]:
model = prepare_model_for_kbit_training(model)

config = LoraConfig(
    # the rank of the adapter, the lower the fewer parameters you'll need to train
    r=8,
    lora_alpha=16, # multiplier, usually 2*r
    bias="none",
    lora_dropout=0.05,
    task_type="CAUSAL_LM",
    # Newer models, such as Phi-3 at time of writing, may require
    # manually setting target modules
    target_modules=['o_proj', 'qkv_proj', 'gate_up_proj', 'down_proj'],
)
model = get_peft_model(model, config)
model


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Phi3ForCausalLM(
      (model): Phi3Model(
        (embed_tokens): Embedding(32064, 3072, padding_idx=32000)
        (embed_dropout): Dropout(p=0.0, inplace=False)
        (layers): ModuleList(
          (0-31): 32 x Phi3DecoderLayer(
            (self_attn): Phi3SdpaAttention(
              (o_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=3072, out_features=3072, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=3072, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=3072, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magn

In [6]:
train_p, tot_p = model.get_nb_trainable_parameters()
print(f'Trainable parameters:      {train_p/1e6:.2f}M')
print(f'Total parameters:          {tot_p/1e6:.2f}M')
print(f'% of trainable parameters: {100*train_p/tot_p:.2f}%')


Trainable parameters:      12.58M
Total parameters:          3833.66M
% of trainable parameters: 0.33%


In [60]:
data = [
    {"prompt": "what is 5 * 3", "completion": "5 * 3 = 15"},
    {"prompt": "What is your name", "completion": "My name is Bismillah."},
    {"prompt": "what is the capital of France", "completion": "The capital of France is Paris."},
    {"prompt": "who wrote Hamlet", "completion": "Hamlet was written by William Shakespeare."},
    {"prompt": "what is the boiling point of water", "completion": "The boiling point of water is 100°C or 212°F at sea level."},
    {"prompt": "what is the square root of 81", "completion": "The square root of 81 is 9."},
    {"prompt": "how many continents are there", "completion": "There are 7 continents on Earth."},
    {"prompt": "what is the chemical symbol for gold", "completion": "The chemical symbol for gold is Au."},
    {"prompt": "who painted the Mona Lisa", "completion": "The Mona Lisa was painted by Leonardo da Vinci."},
    {"prompt": "what is the largest planet in our solar system", "completion": "The largest planet in our solar system is Jupiter."},
    {"prompt": "what is 12 divided by 4", "completion": "12 divided by 4 equals 3."}
]


In [61]:
def format_dataset(examples):
  output_text = []
  for example in examples:
    output_text.append([
        {"role": "user", "content": example["prompt"]},
        {"role": "assistant", "content": example["completion"]}
    ])

  return output_text

In [62]:
formatted = format_dataset(data)

In [63]:
tokenizer = AutoTokenizer.from_pretrained(repo_id)
tokenizer.chat_template


"{% for message in messages %}{% if message['role'] == 'system' %}{{'<|system|>\n' + message['content'] + '<|end|>\n'}}{% elif message['role'] == 'user' %}{{'<|user|>\n' + message['content'] + '<|end|>\n'}}{% elif message['role'] == 'assistant' %}{{'<|assistant|>\n' + message['content'] + '<|end|>\n'}}{% endif %}{% endfor %}{% if add_generation_prompt %}{{ '<|assistant|>\n' }}{% else %}{{ eos_token }}{% endif %}"

In [64]:
print(tokenizer.apply_chat_template(formatted, tokenize=False))


['<|user|>\nwhat is 5 * 3<|end|>\n<|assistant|>\n5 * 3 = 15<|end|>\n<|endoftext|>', '<|user|>\nWhat is your name<|end|>\n<|assistant|>\nMy name is Bismillah.<|end|>\n<|endoftext|>', '<|user|>\nwhat is the capital of France<|end|>\n<|assistant|>\nThe capital of France is Paris.<|end|>\n<|endoftext|>', '<|user|>\nwho wrote Hamlet<|end|>\n<|assistant|>\nHamlet was written by William Shakespeare.<|end|>\n<|endoftext|>', '<|user|>\nwhat is the boiling point of water<|end|>\n<|assistant|>\nThe boiling point of water is 100°C or 212°F at sea level.<|end|>\n<|endoftext|>', '<|user|>\nwhat is the square root of 81<|end|>\n<|assistant|>\nThe square root of 81 is 9.<|end|>\n<|endoftext|>', '<|user|>\nhow many continents are there<|end|>\n<|assistant|>\nThere are 7 continents on Earth.<|end|>\n<|endoftext|>', '<|user|>\nwhat is the chemical symbol for gold<|end|>\n<|assistant|>\nThe chemical symbol for gold is Au.<|end|>\n<|endoftext|>', '<|user|>\nwho painted the Mona Lisa<|end|>\n<|assistant|>\n

In [65]:
tokenizer.pad_token = tokenizer.unk_token
tokenizer.pad_token_id = tokenizer.unk_token_id


In [66]:
sft_config = SFTConfig(
    ## GROUP 1: Memory usage
    # These arguments will squeeze the most out of your GPU's RAM
    # Checkpointing
    gradient_checkpointing=True,    # this saves a LOT of memory
    # Set this to avoid exceptions in newer versions of PyTorch
    gradient_checkpointing_kwargs={'use_reentrant': False},
    # Gradient Accumulation / Batch size
    # Actual batch (for updating) is same (1x) as micro-batch size
    gradient_accumulation_steps=1,
    # The initial (micro) batch size to start off with
    per_device_train_batch_size=16,
    # If batch size would cause OOM, halves its size until it works
    auto_find_batch_size=True,

    ## GROUP 2: Dataset-related
    max_seq_length=64,
    # Dataset
    # packing a dataset means no padding is needed
    packing=True,

    ## GROUP 3: These are typical training parameters
    num_train_epochs=10,
    learning_rate=3e-4,
    # Optimizer
    # 8-bit Adam optimizer - doesn't help much if you're using LoRA!
    optim='paged_adamw_8bit',

    ## GROUP 4: Logging parameters
    logging_steps=10,
    logging_dir='./logs',
    output_dir='./phi3-mini-test-adapter',
    report_to='none'
)


In [67]:
dataset = Dataset.from_list(data)


In [68]:
trainer = SFTTrainer(
    model=model,
    processing_class=tokenizer,
    args=sft_config,
    train_dataset=dataset,
)


Generating train split: 0 examples [00:00, ? examples/s]

/usr/local/lib/python3.11/dist-packages/trl/trainer/sft_trainer.py:403: UserWarning: You passed a processing_class with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `processing_class.padding_side = 'right'` to your code.
  warnings.warn(


In [69]:
trainer.train()


Step,Training Loss
10,0.261000


TrainOutput(global_step=10, training_loss=0.2610464096069336, metrics={'train_runtime': 24.2267, 'train_samples_per_second': 1.651, 'train_steps_per_second': 0.413, 'total_flos': 57372086108160.0, 'train_loss': 0.2610464096069336, 'epoch': 10.0})

In [70]:
def gen_prompt(tokenizer, sentence):
    converted_sample = [{"role": "user", "content": sentence}]
    prompt = tokenizer.apply_chat_template(
        converted_sample, tokenize=False, add_generation_prompt=True
    )
    return prompt


In [89]:
sentence = """
Summarise this:

Skip to main content
ABC News
Just In
For You
Politics
World
Business
Analysis
Sport
Lifestyle
Entertainment
More
Search the ABC
Search…
Start typing for suggestions, press enter to search, or choose from a list of options below.
Log in

Treasurer downplays leaked department document listing possible outcomes of productivity round table
By political reporter Maani Truu
Topic:Economy

3h ago
3 hours ago
Jim Chalmers walks down a corridor.
Treasurer Jim Chalmers says the round table is a "genuine attempt" to tackle big issues in the economy. (ABC News: Ian Cutmore)

In short:
Treasurer Jim Chalmers has downplayed a leaked document from his department that included recommended outcomes of Labor's productivity round table.

He said the briefing was in preparation for the talks and included ideas that had already been put forward, but stressed that the government had not pre-empted any outcomes.

What's next?
Business leaders, unions and economists will gather at Parliament House on Wednesday for the three-day forum.

Link copied

Share article
Treasurer Jim Chalmers has downplayed a leaked document from his department that included possible outcomes of the government's upcoming productivity round table, saying it should not come as a "big surprise" that briefings have been prepared.

The pre-written list of recommended outcomes, prepared for cabinet and seen by the ABC, includes advice to pause changes to the National Construction Code, which is similar to a Coalition proposal that was panned by Labor at the federal election.

Opposition Leader Sussan Ley has seized on the document as evidence that the much-touted talks are a "stitch-up", despite the government's repeated pledge to not rule anything in or out before business leaders, unions and economists gather at Parliament House on Wednesday.

Leaked Treasury advice lists possible outcomes of Labor's yet to be held round table
Photo shows Jim Chalmers in press conferenceJim Chalmers in press conference
The leaked advice reveals what the government expects to achieve at next week's productivity round table, recommending a freeze of the National Construction Code, and an AI plan to cut environmental red tape.

"When I read information that says it's all been choreographed, it's all been lined up, even to the level of announcements being made from outcomes, I wonder whether people who are attending this round table are indeed wasting their time," Ms Ley told reporters in Adelaide on Thursday.

The treasurer, however, was adamant that the forum was a "genuine attempt" to tackle big issues in the economy and that the government did not "pre-empt any of the reform directions" that would come out of it.

"I don't think it should come as a big surprise to anyone that the Treasury has been briefing us on those ideas, helping us to prepare for those proposals that people have said that they will put to us," he said.

Recommendations not government policy, Chalmers says
The Treasury advice, first reported by the ABC on Thursday, also included measures to speed up housing approvals, such as a national artificial intelligence plan to cut environmental red tape, and reforms to clear a backlog of 30,000 housing approvals currently being assessed under the Environment Protection and Biodiversity Act.

Mr Chalmers would not be drawn on on whether the government would accept the recommendation to pause changes to the National Construction Code, telling reporters "there are a number of issues that have been proposed to us".

He also stressed that the document was "not a government announcement" and that the recommendations within it were not government policy.

Will Labor's conversation about everything produce anything much?
Photo shows Anthony Albanese and Jim Chalmers in the House of Representatives.Anthony Albanese and Jim Chalmers in the House of Representatives.
Inside Labor, two camps have formed around the government's productivity round table: those who doubt Anthony Albanese has the inclination to tackle contentious new policy and those who see a genuine opportunity to achieve something of substance.

"I have been consulting for weeks now in probably one of the most intensive periods of consultation that people have seen for a long time and ideas like that have been put to us, not just those ideas that have been reported today but probably hundreds of different ideas have been put to us," he said.

Housing Minister Clare O'Neil confirmed the code would be up for discussion at the talks, but echoed the treasurer in stating she did not want to pre-empt any outcomes.

Industry has approached next week's summit cautiously, after some business leaders left the government's 2022 Jobs and Skills Summit feeling the government had entered with a pre-determined outcome.

Mr Chalmers has repeatedly said nothing would be off the table at the talks, only specifying that any proposal should be budget positive or at least budget neutral.

He has previously flagged that tax reform would be a priority, but that was thrown into question after the prime minister last week suggested the government was "only" looking at tax policies it took to the election.

Earlier on Thursday, Prime Minister Anthony Albanese also batted away accusations the meeting was a waste of time, saying "ideas are popping up all the time".

"Those ideas are getting assessed as they're being made. Government will make decisions, to be very clear, governments make decisions," he said.

"What next week is, though, is an opportunity for people to advance their ideas, to advance policies, and that's a really constructive thing."

Get a morning news briefing in your inbox
Subscribe to our ABC NewsMail bringing you breaking news from overnight, plus a heads-up on the stories that will shape your day.
Email address
Enter your email address
Subscribe
Your information is being handled in accordance with the ABC Privacy Collection Statement.
This site is protected by reCAPTCHA and the Google Privacy Policy and Terms of Service apply.

Posted 3h ago3 hours ago

Share
Top Stories
'Giant in his field': Legendary film critic David Stratton dead at 85
Topic:Arts, Culture and Entertainment

Photo shows An older man in a checked shirt and vest sits on a bench outside an old Australian cinema.An older man in a checked shirt and vest sits on a bench outside an old Australian cinema.
Israel's 'trust us' mentality loses power as it targets journalists
A man in a suit stands in a corridor of parliament house.
Analysis by Matthew Doran

Photo shows A crowd of men carry two covered bodies in stretchers past ruined buildings.A crowd of men carry two covered bodies in stretchers past ruined buildings.
Julie Bishop stays put and Labor abandons the P-word
Brett Worthington stands in a courtyard inside parliament house
Analysis by Brett Worthington

Photo shows Jim Chalmers and Anthony Albanese laugh at a pub in BrisbaneJim Chalmers and Anthony Albanese laugh at a pub in Brisbane
Head of NSW child abuse squad warns of 'red flags' in childcare centres
Topic:Child Care

Photo shows Kids toys on the floor.Kids toys on the floor.
How Putin has used flattery and deception to 'fool' five US presidents

Analysis by John Lyons

Photo shows Two men leaning towards each other to chatTwo men leaning towards each other to chat
Related stories
Peak union body calls for four-day work week
Topic:Work

Photo shows Kitchen worker uses tongs in a pub kitchen in Sydney.Kitchen worker uses tongs in a pub kitchen in Sydney.
Welfare groups join push to tackle negative gearing, CGT concessions
Topic:Tax

Photo shows A close up of Cassandra Goldie in parliament house.A close up of Cassandra Goldie in parliament house.
Related topics
Australia

Economy

Federal Government

Federal Parliament

Government and Politics

Top Stories
'Giant in his field': Legendary film critic David Stratton dead at 85
Topic:Arts, Culture and Entertainment

Photo shows An older man in a checked shirt and vest sits on a bench outside an old Australian cinema.An older man in a checked shirt and vest sits on a bench outside an old Australian cinema.
Israel's 'trust us' mentality loses power as it targets journalists
A man in a suit stands in a corridor of parliament house.
Analysis by Matthew Doran

Julie Bishop stays put and Labor abandons the P-word
Brett Worthington stands in a courtyard inside parliament house
Analysis by Brett Worthington

Head of NSW child abuse squad warns of 'red flags' in childcare centres
Topic:Child Care

How Putin has used flattery and deception to 'fool' five US presidents

Analysis by John Lyons

Just In
David Stratton's iconic reviews, from favourites to 'scathing' critiques
Topic:Arts, Culture and Entertainment

2m ago
2 minutes ago
Search continues for missing couple and light plane, despite no trace
Topic:Air Incidents

23m ago
23 minutes ago
Gun owners protest laws as police minister warns of 'firearm anarchy'
Topic:State and Territory Government

14m ago
14 minutes ago
Bhutanese community rallies in desperate search for missing student
Topic:Missing Person

36m ago
36 minutes ago
More Just In
Back to top
Footer
ABC News homepage
More From ABC NEWS
We acknowledge Aboriginal and Torres Strait Islander peoples as the First Australians and Traditional Custodians of the lands where we live, learn, and work.

Sections
ABC NEWS
Just In
Watch Live
Politics
World
Business
Analysis
Sport
Science
Health
Entertainment
Lifestyle
Fact Check
Other
News in language
中文
Berita Bahasa Indonesia
Tok Pisin
Connect with ABC News
Facebook
Instagram
YouTube
Apple News
X (formerly Twitter)
More from ABC News
Contact ABC NEWS
This service may include material from Agence France-Presse (AFP), APTN, Reuters, AAP, CNN and the BBC World Service which is copyright and cannot be reproduced.

AEST = Australian Eastern Standard Time which is 10 hours ahead of GMT (Greenwich Mean Time)

Editorial Policies
Accessibility
Help
Contact Us
About the ABC
Privacy Policy
Terms of Use
© 2025 ABC
"""
prompt = gen_prompt(tokenizer, sentence)
print(prompt)


<|user|>

Summarise this:

Skip to main content
ABC News
Just In
For You
Politics
World
Business
Analysis
Sport
Lifestyle
Entertainment
More
Search the ABC
Search…
Start typing for suggestions, press enter to search, or choose from a list of options below.
Log in

Treasurer downplays leaked department document listing possible outcomes of productivity round table
By political reporter Maani Truu
Topic:Economy

3h ago
3 hours ago
Jim Chalmers walks down a corridor. 
Treasurer Jim Chalmers says the round table is a "genuine attempt" to tackle big issues in the economy. (ABC News: Ian Cutmore)

In short:
Treasurer Jim Chalmers has downplayed a leaked document from his department that included recommended outcomes of Labor's productivity round table.

He said the briefing was in preparation for the talks and included ideas that had already been put forward, but stressed that the government had not pre-empted any outcomes.

What's next?
Business leaders, unions and economists will gather at

In [87]:
def generate(model, tokenizer, prompt, max_new_tokens=64, skip_special_tokens=False):
    tokenized_input = tokenizer(
        prompt, add_special_tokens=False, return_tensors="pt"
    ).to(model.device)

    model.eval()
    gen_output = model.generate(**tokenized_input,
                                eos_token_id=tokenizer.eos_token_id,
                                max_new_tokens=max_new_tokens)

    output = tokenizer.batch_decode(gen_output, skip_special_tokens=skip_special_tokens)
    return output[0]


In [88]:
print(generate(model, tokenizer, prompt))


<|user|> Skip to main content
ABC News
Just In
For You
Politics
World
Business
Analysis
Sport
Lifestyle
Entertainment
More
Search the ABC
Search…
Start typing for suggestions, press enter to search, or choose from a list of options below.
Log in

Treasurer downplays leaked department document listing possible outcomes of productivity round table
By political reporter Maani Truu
Topic:Economy

3h ago
3 hours ago
Jim Chalmers walks down a corridor. 
Treasurer Jim Chalmers says the round table is a "genuine attempt" to tackle big issues in the economy. (ABC News: Ian Cutmore)

In short:
Treasurer Jim Chalmers has downplayed a leaked document from his department that included recommended outcomes of Labor's productivity round table.

He said the briefing was in preparation for the talks and included ideas that had already been put forward, but stressed that the government had not pre-empted any outcomes.

What's next?
Business leaders, unions and economists will gather at Parliament House 